# McNemar test

In [1]:
!pip install statsmodels==0.13.5

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np                     # Llibreria matematica
import matplotlib.pyplot as plt        # Per mostrar plots
import sklearn                         # Llibreia de DM
import sklearn.datasets as ds          # Per carregar més facilment el dataset digits
import sklearn.model_selection as cv   # Pel Cross-validation
import sklearn.neighbors as nb         # Per fer servir el knn
import pandas as pd
from sklearn.model_selection import cross_val_predict, train_test_split

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [3]:
data = pd.read_csv('fake_job_preprocessed.csv')
data

,title,location,department,salary_range,company_profile,description,requirements,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,6.333616e-09,0.041026,0.000000e+00,2.222222e-02,0.000051,0.056281,0.056281,0,1,0,0.166667,0.068627,0.102041,4.431315e-03,0.017964,0
1,3.003079e-01,0.380666,6.413994e-02,5.628121e-02,0.144389,0.056281,0.015136,0,0,0,0.041833,0.039746,0.012579,3.703704e-02,0.032536,0
2,5.628121e-02,0.000000,5.628121e-02,1.513635e-02,0.015136,0.056281,0.056281,0,1,1,0.050260,0.030513,0.027523,0.000000e+00,0.019231,0
3,1.513635e-02,0.002669,6.413994e-02,1.160042e-10,0.144389,0.056281,0.056281,0,0,0,0.050260,0.030513,0.017430,1.732102e-02,0.015110,0
4,5.628121e-02,0.000003,3.430079e-02,1.391622e-04,0.015136,0.056281,0.056281,0,1,1,0.050260,0.145455,0.017430,7.816336e-13,0.017516,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5201,5.628121e-02,0.003155,3.430079e-02,3.153322e-10,0.000019,0.056281,0.056281,0,1,1,0.050260,0.132733,0.017430,1.911765e-01,0.017516,0
5202,3.806656e-01,0.056281,3.430079e-02,3.003079e-01,0.144389,0.056281,0.380666,0,1,0,0.050260,0.132733,0.065969,1.276596e-01,0.017516,0
5203,0.000000e+00,0.021277,2.272727e-02,0.000000e+00,0.000377,0.056281,0.000377,0,1,0,0.050260,0.039746,0.017430,1.879699e-03,0.000000,0
5204,6.944797e-06,0.057692,1.391622e-04,3.749082e-17,0.144389,0.002669,0.002669,0,0,0,0.050260,0.030513,0.017430,4.431315e-03,0.015110,0


In [4]:
X = data.values[:, 0:data.shape[1]-1]
y = data.values[:, data.shape[1]-1]

print(X.shape)

# Let's do a simple cross-validation: split data into training and test sets (test 30% of data)
(X_train, X_test,  y_train, y_test) = cv.train_test_split(X, y, test_size=.33, random_state=1)

(5206, 15)


In [9]:
cv = StratifiedKFold(n_splits=10)
# Naïve Bayes classifier
naiv_b = GaussianNB()

# KNN classifier
knn = nb.KNeighborsClassifier(n_neighbors=5)

# Decision Tree classifier
BestPar = {'splitter': 'best', 'min_samples_split': 5, 'max_features': 7, 'max_depth': 5, 'criterion': 'entropy'}
dt = tree.DecisionTreeClassifier(criterion=BestPar['criterion'], max_depth=BestPar['max_depth'],
                                max_features=BestPar['max_features'],
                                min_samples_split=BestPar['min_samples_split'], splitter=BestPar['splitter'])

# Support Vector Machine classifier
svm = SVC(kernel='linear', C=100)

# Majority voting
BestPar = {'splitter': 'best', 'min_samples_split': 5, 'max_features': 7, 'max_depth': 5, 'criterion': 'entropy'}
# Crear lista de modelos
modelos = [('nb', GaussianNB()), 
           ('knn', nb.KNeighborsClassifier(n_neighbors=5)),
           ('dt', tree.DecisionTreeClassifier(criterion=BestPar['criterion'], max_depth=BestPar['max_depth'],
                                max_features=BestPar['max_features'],
                                min_samples_split=BestPar['min_samples_split'], splitter=BestPar['splitter'])),
            ('svc', SVC(kernel='linear', C=100))]

# Majority voting classifier
majority_voting = VotingClassifier(estimators=modelos, voting='hard')

# Bagging classifier
bagging = BaggingClassifier(base_estimator=tree.DecisionTreeClassifier(),n_estimators=200,max_features=0.75)

# Random Forest
random_forest = RandomForestClassifier(n_estimators=50)

# AdaBoost
ada_boost = AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth=5),n_estimators=100)

In [10]:
y_pred_nb = cross_val_predict(naiv_b, X=X_test, y=y_test,  cv=cv)
y_pred_knn = cross_val_predict(knn, X=X_test, y=y_test,  cv=cv)
y_pred_dt = cross_val_predict(dt, X=X_test, y=y_test,  cv=cv)
y_pred_svm = cross_val_predict(svm, X=X_test, y=y_test,  cv=cv)
y_pred_majority_voting = cross_val_predict(majority_voting, X=X_test, y=y_test,  cv=cv)
y_pred_bagging = cross_val_predict(bagging, X=X_test, y=y_test,  cv=cv)
y_pred_random_forest = cross_val_predict(random_forest, X=X_test, y=y_test,  cv=cv)
y_pred_ada_boost = cross_val_predict(ada_boost, X=X_test, y=y_test,  cv=cv)

In [14]:
from statsmodels.stats.contingency_tables import mcnemar

# Calculamos las matrices de confusión para cada modelo
confusion_matrix_model_1 = confusion_matrix(y_test, y_pred_nb)
confusion_matrix_model_2 = confusion_matrix(y_test, y_pred_knn)
confusion_matrix_model_3 = confusion_matrix(y_test, y_pred_dt)
confusion_matrix_model_4 = confusion_matrix(y_test, y_pred_svm)
confusion_matrix_model_5 = confusion_matrix(y_test, y_pred_majority_voting)
confusion_matrix_model_6 = confusion_matrix(y_test, y_pred_bagging)
confusion_matrix_model_7 = confusion_matrix(y_test, y_pred_random_forest)
confusion_matrix_model_8 = confusion_matrix(y_test, y_pred_ada_boost)

# Establecemos el valor de significación del 5%, que se usa generalmente
alpha = 0.05

# Realizamos la prueba de McNemar
# Comparativa del modelo Naïve Bayes con el resto de modelos
result1_2 = mcnemar(confusion_matrix_model_1, confusion_matrix_model_2.all())
result1_3 = mcnemar(confusion_matrix_model_1, confusion_matrix_model_3.all())
result1_4 = mcnemar(confusion_matrix_model_1, confusion_matrix_model_4.all())
result1_5 = mcnemar(confusion_matrix_model_1, confusion_matrix_model_5.all())
result1_6 = mcnemar(confusion_matrix_model_1, confusion_matrix_model_6.all())
result1_7 = mcnemar(confusion_matrix_model_1, confusion_matrix_model_7.all())
result1_8 = mcnemar(confusion_matrix_model_1, confusion_matrix_model_8.all())
print('Naïve Bayes con KNN:\n', result1_2)
print('Naïve Bayes con DecisionTree:\n', result1_3)
print('Naïve Bayes con SVM:\n', result1_4)
print('Naïve Bayes con MajorityVoting:\n', result1_5)
print('Naïve Bayes con Bagging:\n', result1_6)
print('Naïve Bayes con RandomForest:\n', result1_7)
print('Naïve Bayes con AdaBoost:\n', result1_8)

print()

# Comparativa del modelo KNN con el resto de modelos
result2_3 = mcnemar(confusion_matrix_model_2, confusion_matrix_model_3.all())
result2_4 = mcnemar(confusion_matrix_model_2, confusion_matrix_model_4.all())
result2_5 = mcnemar(confusion_matrix_model_2, confusion_matrix_model_5.all())
result2_6 = mcnemar(confusion_matrix_model_2, confusion_matrix_model_6.all())
result2_7 = mcnemar(confusion_matrix_model_2, confusion_matrix_model_7.all())
result2_8 = mcnemar(confusion_matrix_model_2, confusion_matrix_model_8.all())
print('KNN con DecisionTree:\n', result2_3)
print('KNN con SVM:\n', result2_4)
print('KNN con MajorityVoting:\n', result2_5)
print('KNN con Bagging:\n', result2_6)
print('KNN con RandomForest:\n', result2_7)
print('KNN con AdaBoost:\n', result2_8)

print()

# Comparativa del modelo DecisionTree con el resto de modelos
result3_4 = mcnemar(confusion_matrix_model_3, confusion_matrix_model_4.all())
result3_5 = mcnemar(confusion_matrix_model_3, confusion_matrix_model_5.all())
result3_6 = mcnemar(confusion_matrix_model_3, confusion_matrix_model_6.all())
result3_7 = mcnemar(confusion_matrix_model_3, confusion_matrix_model_7.all())
result3_8 = mcnemar(confusion_matrix_model_3, confusion_matrix_model_8.all())
print('DecisionTree con SVM:\n', result3_4)
print('DecisionTree con MajorityVoting:\n', result3_5)
print('DecisionTree con Bagging:\n', result3_6)
print('DecisionTree con RandomForest:\n', result3_7)
print('DecisionTree con AdaBoost:\n', result3_8)

print()

# Comparativa del modelo SVM con el resto de modelos
result4_5 = mcnemar(confusion_matrix_model_4, confusion_matrix_model_5.all())
result4_6 = mcnemar(confusion_matrix_model_4, confusion_matrix_model_6.all())
result4_7 = mcnemar(confusion_matrix_model_4, confusion_matrix_model_7.all())
result4_8 = mcnemar(confusion_matrix_model_4, confusion_matrix_model_8.all())
print('SVM con MajorityVoting:\n', result4_5)
print('SVM con Bagging:\n', result4_6)
print('SVM con RandomForest:\n', result4_7)
print('SVM con AdaBoost:\n', result4_8)

print()

# Comparativa del modelo MajorityVoting con el resto de modelos
result5_6 = mcnemar(confusion_matrix_model_5, confusion_matrix_model_6.all())
result5_7 = mcnemar(confusion_matrix_model_5, confusion_matrix_model_7.all())
result5_8 = mcnemar(confusion_matrix_model_5, confusion_matrix_model_8.all())
print('MajorityVoting con Bagging:\n', result5_6)
print('MajorityVoting con RandomForest:\n', result5_7)
print('MajorityVoting con AdaBoost:\n', result5_8)

print()

# Comparativa del modelo Bagging con el resto de modelos
result6_7 = mcnemar(confusion_matrix_model_6, confusion_matrix_model_7.all())
result6_8 = mcnemar(confusion_matrix_model_6, confusion_matrix_model_8.all())
print('Bagging con RandomForest:\n', result6_7)
print('Bagging con AdaBoost:\n', result6_8)

print()

# Comparativa del modelo RandomForest con AdaBoost
result7_8 = mcnemar(confusion_matrix_model_6, confusion_matrix_model_8.all())
print('RandomForest con AdaBoost:\n', result7_8)

Naïve Bayes con KNN:
 pvalue      1.802802378388091e-08
statistic   9.0
Naïve Bayes con DecisionTree:
 pvalue      1.802802378388091e-08
statistic   9.0
Naïve Bayes con SVM:
 pvalue      1.802802378388091e-08
statistic   9.0
Naïve Bayes con MajorityVoting:
 pvalue      1.802802378388091e-08
statistic   9.0
Naïve Bayes con Bagging:
 pvalue      1.802802378388091e-08
statistic   9.0
Naïve Bayes con RandomForest:
 pvalue      1.802802378388091e-08
statistic   9.0
Naïve Bayes con AdaBoost:
 pvalue      1.802802378388091e-08
statistic   9.0

KNN con DecisionTree:
 pvalue      2.4904030624384174e-05
statistic   9.0
KNN con SVM:
 pvalue      2.4904030624384174e-05
statistic   9.0
KNN con MajorityVoting:
 pvalue      2.4904030624384174e-05
statistic   9.0
KNN con Bagging:
 pvalue      2.4904030624384174e-05
statistic   9.0
KNN con RandomForest:
 pvalue      2.4904030624384174e-05
statistic   9.0
KNN con AdaBoost:
 pvalue      2.4904030624384174e-05
statistic   9.0

DecisionTree con SVM:
 pvalu

En la mayoría de los casos, el p-valor es menor al valor significativo que hemos establecido, por lo que podemos afirmar que existe una diferencia significativa entre todos los modelos. Excepto entre DT y SVM, SVM y Majority Voting y todos los meta-metodos entre si.En estos el p-value es mayor, y por lo tanto no podemos afirmar diferencia.

Respecto la statistica de McNemar, como no es igual a cero en nigun caso, podemos concluir que existe una diferencia significativa entre todos los modelos en términos de TP y FN.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=91572d65-ca52-49ec-8072-8fb246390be4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>